In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Activation, Conv2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Input, MaxPooling2D, SeparableConv2D
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import os
import pandas as pd

https://github.com/kumarnikhil936/face_emotion_recognition_cnn<br>
https://github.com/XiuweiHe/EmotionClassifier/blob/master/src/cnn.py --> consider doing a tiny ALEXNET

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
train_dir = './data/train'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   validation_split=0.2,
                                   rotation_range=5,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1. / 255,
                                   validation_split=0.2)

In [ ]:
train_dataset = train_datagen.flow_from_directory(directory=train_dir,
                                                  target_size=(48, 48),
                                                  class_mode='categorical',
                                                  subset='training',
                                                  batch_size=64)

In [ ]:
valid_dataset = valid_datagen.flow_from_directory(directory=train_dir,
                                                  target_size=(48, 48),
                                                  class_mode='categorical',
                                                  subset='validation',
                                                  batch_size=64)

In [ ]:
image_shape = (48, 48, 3)
num_classes = 7
epochs = 50
lr = 1e-10
regularization = l2(0.01)

model

In [ ]:
image_input = Input(image_shape)
x = Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(
    image_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(filters=8, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

module 1<br>
residual module

In [ ]:
residual = Conv2D(filters=16, kernel_size=(1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

In [ ]:
x = SeparableConv2D(filters=16, kernel_size=(3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(
    x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=16, kernel_size=(3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(
    x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

module 2<br>
residual module

In [ ]:
residual = Conv2D(filters=32, kernel_size=(1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

In [ ]:
x = SeparableConv2D(filters=32, kernel_size=(3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(
    x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(
    x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

module 3<br>
residual module

In [ ]:
residual = Conv2D(filters=64, kernel_size=(1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

In [ ]:
x = SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(
    x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(
    x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

module 4<br>
residual module

In [ ]:
residual = Conv2D(filters=128, kernel_size=(1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

In [ ]:
x = SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(
    x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(
    x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

In [ ]:
x = Conv2D(filters=num_classes, kernel_size=(3, 3), padding='same')(x)
x = GlobalAveragePooling2D()(x)

In [ ]:
output = Activation('softmax', name='predictions')(x)
model = Model(image_input, output)

In [ ]:
model.summary()

In [ ]:
lrd = ReduceLROnPlateau(monitor='val_loss', patience=20, verbose=1, factor=0.50, min_lr=lr)

In [ ]:
mcp = ModelCheckpoint('./weights/miniXception_model.h5', monitor='val_accuracy',
                      mode='max',
                      save_best_only=True)

In [ ]:
es = EarlyStopping(verbose=1, patience=20)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_dataset, validation_data=valid_dataset, epochs=epochs,
                    verbose=1, callbacks=[lrd, mcp, es])

In [ ]:
os.makedirs('./history', exist_ok=True)
file_name = './history/miniXception_hist.csv'
with open(file_name, mode='w') as f:
    pd.DataFrame(history.history).to_csv(f)